In [1]:
import pandas as pd
from pathlib import Path

In [2]:
btc_path = Path('../Projects/btwTweetDf')

btc_df = pd.read_csv(btc_path)

In [3]:
btc_df.drop(['Unnamed: 0'], axis=1, inplace = True)

In [4]:
btc_df.head()

,date,tweet,hashtags,name,nlikes,nreplies,nretweets
0,2016-01-12 07:05:27,As Netflix expands its international subscribe...,['bitcoin'],Investors.com,1,0,1
1,2017-12-29 08:12:07,Is Gold Getting Ready To Shine Again As Bitcoi...,[],Investors.com,2,1,1
2,2017-12-28 22:34:02,"Apple Apologizes, Bitcoin Rebounds: S&amp;P 50...",[],Investors.com,3,1,4
3,2017-12-28 13:43:09,Stocks Hold Modest Gains; Netflix Snaps Back; ...,[],Investors.com,1,1,1
4,2017-12-28 10:00:21,"Thursday's Top Stories: Apple, Amazon are each...",[],Investors.com,2,0,1


In [5]:
import os
from pathlib import Path
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
get_ipython().run_line_magic("matplotlib", "inline")

In [6]:
btc_df.sort_values(by = 'date')
btc_df

,date,tweet,hashtags,name,nlikes,nreplies,nretweets
0,2016-01-12 07:05:27,As Netflix expands its international subscribe...,['bitcoin'],Investors.com,1,0,1
1,2017-12-29 08:12:07,Is Gold Getting Ready To Shine Again As Bitcoi...,[],Investors.com,2,1,1
2,2017-12-28 22:34:02,"Apple Apologizes, Bitcoin Rebounds: S&amp;P 50...",[],Investors.com,3,1,4
3,2017-12-28 13:43:09,Stocks Hold Modest Gains; Netflix Snaps Back; ...,[],Investors.com,1,1,1
4,2017-12-28 10:00:21,"Thursday's Top Stories: Apple, Amazon are each...",[],Investors.com,2,0,1
...,...,...,...,...,...,...,...
1189,2021-08-19 07:45:49,Not another crypto hack....$65 million was sto...,"['coinmarketcap', 'crypto', 'cryptocurrency', ...",CoinMarketCap,254,46,61
1190,2021-08-19 02:50:38,Market Speculations: Can Bitcoin Maintain Its ...,"['coinmarketcap', 'alexandria', 'bitcoin', 'btc']",CoinMarketCap,337,69,136
1191,2021-08-18 17:00:15,"Join the ""SWAPP Protocol ($SWAPP)"" Airdrop ht...","['coinmarketcap', 'alexandria', 'crypto', 'cry...",CoinMarketCap,554,128,306
1192,2021-08-18 15:56:00,"Join the ""PolkaBridge ($PBR)"" Airdrop https:/...","['coinmarketcap', 'alexandria', 'crypto', 'cry...",CoinMarketCap,473,104,252


In [7]:
btc_df.to_csv('BTC_Tweets_16_21')

In [8]:
#Download Vader/Update the VADER Lexicon
nltk.download("vader_lexicon")

#Initiate the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\n1ck_\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [10]:
# Sentiment scores dictionaries
tweet_sent = {
    "tweet_compound": [],
    "tweet_pos": [],
    "tweet_neu": [],
    "tweet_neg": [],
    "tweet_sent": [],

}

# Get sentiment for the text and the title
for index, row in btc_df.iterrows():
    try:
        # Sentiment scoring with VADER
        tweet_sentiment = analyzer.polarity_scores(row["tweet"])
        tweet_sent["tweet_compound"].append(tweet_sentiment["compound"])
        tweet_sent["tweet_pos"].append(tweet_sentiment["pos"])
        tweet_sent["tweet_neu"].append(tweet_sentiment["neu"])
        tweet_sent["tweet_neg"].append(tweet_sentiment["neg"])
        tweet_sent["tweet_sent"].append(get_sentiment(tweet_sentiment["compound"]))

    
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
tweet_sentiment_df = pd.DataFrame(tweet_sent)
tweet_sentiment_df

,tweet_compound,tweet_pos,tweet_neu,tweet_neg,tweet_sent
0,0.0000,0.083,0.833,0.083,0
1,0.3612,0.172,0.828,0.000,1
2,0.3612,0.238,0.762,0.000,1
3,0.3400,0.167,0.833,0.000,1
4,0.2732,0.080,0.888,0.032,1
...,...,...,...,...,...
1189,-0.5411,0.000,0.851,0.149,-1
1190,0.0000,0.000,1.000,0.000,0
1191,0.6486,0.194,0.806,0.000,1
1192,0.6486,0.202,0.798,0.000,1


In [11]:
def join_dataframes(df1,df2): 
    df1=df1.join(df2)
    return df1

In [12]:
tweet_df=join_dataframes(btc_df, tweet_sentiment_df)
tweet_df.columns

Index(['date', 'tweet', 'hashtags', 'name', 'nlikes', 'nreplies', 'nretweets',
       'tweet_compound', 'tweet_pos', 'tweet_neu', 'tweet_neg', 'tweet_sent'],
      dtype='object')

In [13]:
tweet_df.drop(['hashtags', 'nlikes', 'nreplies', 'nretweets'], axis = 1, inplace = True)

In [14]:
tweet_df['date'] = pd.to_datetime(tweet_df['date'], format ='%Y-%m-%d %H:%M:%S')
tweet_df.head()

,date,tweet,name,tweet_compound,tweet_pos,tweet_neu,tweet_neg,tweet_sent
0,2016-01-12 07:05:27,As Netflix expands its international subscribe...,Investors.com,0.0000,0.083,0.833,0.083,0
1,2017-12-29 08:12:07,Is Gold Getting Ready To Shine Again As Bitcoi...,Investors.com,0.3612,0.172,0.828,0.000,1
2,2017-12-28 22:34:02,"Apple Apologizes, Bitcoin Rebounds: S&amp;P 50...",Investors.com,0.3612,0.238,0.762,0.000,1
3,2017-12-28 13:43:09,Stocks Hold Modest Gains; Netflix Snaps Back; ...,Investors.com,0.3400,0.167,0.833,0.000,1
4,2017-12-28 10:00:21,"Thursday's Top Stories: Apple, Amazon are each...",Investors.com,0.2732,0.080,0.888,0.032,1


In [16]:
import hvplot.pandas
tweet_df.sort_values('date').hvplot.line(
"date",
"tweet_compound",
xlabel="Date",
ylabel="Sentiment Score",
groupby="name",
title="BTC Twitter Sentiment")

:DynamicMap   [name]
   :Curve   [date]   (tweet_compound)